In [2]:
pip install markovify

     |████████████████████████████████| 238 kB 3.8 MB/s eta 0:00:01
  Created wheel for markovify: filename=markovify-0.8.0-py3-none-any.whl size=10694 sha256=d15180bb7b207611978685810702fdd396fc6d071898612deed59109d74256e8
  Stored in directory: /Users/vilandao/Library/Caches/pip/wheels/05/96/15/0ded02d3a0dddbc64202c8206791bda22af3d90e0fc1eaf056
Successfully built markovify
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import libraries:
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine
import spacy
import markovify

import warnings
warnings.filterwarnings(action="ignore")

!python -m spacy download en

     |████████████████████████████████| 11.1 MB 897 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=7b3207dea4680ec25117f593eb6128a723f002f3c6e16ffdf7b71d175d37603d
  Stored in directory: /private/var/folders/qn/pvcvvqz50ys5skppg1_5zj280000gn/T/pip-ephem-wheel-cache-y41oo637/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/vilandao/opt/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/vilandao/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
# Connect to the database to pull the data
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

# Create engine to connect to the dataset
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

tweets_df = pd.read_sql_query('select * from twitter',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [6]:
nlp = spacy.load('en')

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
twitter_doc = nlp(" ".join(tweets_df.text))

### Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences

In [11]:
# all the processing work is done below, so it may take a while
twitter_negative_doc = nlp(" ".join(tweets_df[tweets_df["airline_sentiment"]=="negative"].text))

tweet_negative_sents = " ".join([sent.text for sent in twitter_negative_doc.sents if len(sent.text) > 1])

tweet_negative_generator = markovify.Text(tweet_negative_sents, state_size = 3)

# 20 randomly generated negative sentences
for i in range(20):
    print(tweet_negative_generator.make_sentence(tries=100))

# 20 randomly-generated negative sentences of no more than 100 characters
for i in range(20):
    print(tweet_negative_generator.make_short_sentence(100, tries=100))

@AmericanAir I want the flight I have why don't you.
@AmericanAir rude gate agent at arrival MKE 2/23.
I only flew them because I had loads of time running to my connecting flight will need.
@United maybe find a different gate was rude when I asked to many questions. bag might still be on flight 413.
@united your ground crew was so SLOW.
@USAirways I've been waiting for over 10 mins...
@JetBlue I am heading to JFK now and put in.
I've now been on hold 2 hours 54 minutes.
For an hour and a half which is longer than @ABCNetwork's #Oscars coverage.
@AmericanAir but seriously if my cats dead I'm going to start charging you for the bad service and bad Friday!
@USAirways not impressed with your customer service handler who misconnected me.
@JetBlue I don't have a good sense on what's happening with 1217 Phl to LAX?
I was on UA3782 and it was the most reliable..
@USAirways hey!! Yeah last time I did, you screwed me out of my account.
I've been on hold for over a hour trying to see a single CS

### Do the previous task this time bu using only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.

In [9]:
# all the processing work is done below, so it may take a while
twitter_positive_doc = nlp(" ".join(tweets_df[tweets_df["airline_sentiment"]=="positive"].text))

tweet_positive_sents = " ".join([sent.text for sent in twitter_positive_doc.sents if len(sent.text) > 1])

tweet_positive_generator = markovify.Text(tweet_positive_sents, state_size = 3)

# 20 randomly generated sentences
for i in range(20):
    print(tweet_positive_generator.make_sentence(tries=100))

# 20 randomly-generated sentences of no more than 100 characters
for i in range(20):
    print(tweet_positive_generator.make_short_sentence(100, tries=100))

It was wind and I was present for her help.
@united thanks for the recommendation.
We were delayed but your staff and alerts have been helpful.
@united we were able to help out!
Need it on my spotify stat #guiltypleasures @VirginAmerica @ladygaga @carrieunderwood love all three but you really can't beat the classics!
Her flight leaves at 2 but she's arriving to the airport the rebooked ticket was refunded.
@united well, thanks for not getting my BusinessFirst priority tagged bag onto my connecting flight for me so I wouldn't miss it.
@united This is probably the least dependable airline in the world @JetBlue I would go to Las Vegas.
Not #unreal @USAirways thanks for the reply.
@USairways Since I am a raving fan.
@USAirways knows customer service!! Thank you for the hard work and making it happen.
#flyfi http://t.co/8jceDiKY9U @JetBlue thanks @JetBlue Then en route to LHR @united thank you!
Thanks for the response - when is the next flight @USAirways You all work hard at making sure thi

### This time train your Markov chain model using all the tweets and generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [10]:
# all the processing work is done below, so it may take a while
twitter_doc = nlp(" ".join(tweets_df.text))

tweet_sents = " ".join([sent.text for sent in twitter_doc.sents if len(sent.text) > 1])

tweet_generator = markovify.Text(tweet_sents, state_size = 3)

# three randomly generated sentences
for i in range(20):
    print(tweet_generator.make_sentence(tries=100))

# three randomly-generated sentences of no more than 100 characters
for i in range(20):
    print(tweet_generator.make_short_sentence(100, tries=100))

#usairwayssucks @USAirways after missing my flight and now are still delayed...
Over 20 minutes waiting just to get you on your way ASAP @united @RobBogart After I waited hours in the past week.
Flight to Dallas was Cancelled Flightled http://t.co/YiwLhQhZgp @USAirways, 1st a reaccommodation, now a delayed flight at this very moment.
Get error msg when trying to purchase a flight that is this week!!!!! @AmericanAir You guys are a huge joke and Cancelled Flight as members w u.
Door was closed only to be told I need to Cancelled Flight one leg of a flight, but I've since learned @Delta had one!
@united So disappointed in the service and cheap upgrades American Provides!
#missedflight back to Jet Blue @SouthwestAir when are you guys going to leave the airport w/o my luggage.
I hope you all end up on hold for 1 hour.
@USAirways @AmericanAir nvm my bad it's been an hour @SouthwestAir departing MDW it's cold 26F, windy with snow on the ground instead of diverting the plane &amp; adding 2 hrs